### Cascading Classification

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

AUTOTUNE = tf.data.experimental.AUTOTUNE
import IPython.display as display


import os
from os import listdir
from os.path import isfile, join
import pickle
import random
#importing module
import sys
sys.path.insert(0, '../data')
from datahandler_cascading import create_dataset

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


In [2]:
with open('../data/filenames.pkl', 'rb') as infile:
    filenames = pickle.load(infile)

with open('../data/labels.pkl', 'rb') as infile2:
    labels = pickle.load(infile2)
    
print(len(filenames), len(labels))
df = pd.concat([pd.Series(filenames, name='filenames'), pd.Series(labels, name='labels')], axis=1)

24999 24999


In [3]:
with open('../data/filenames_level2.pkl', 'rb') as infile:
    filenames2 = pickle.load(infile)

with open('../data/labels_level2.pkl', 'rb') as infile2:
    labels2 = pickle.load(infile2)

print(len(filenames2), len(labels2))
df2 = pd.concat([pd.Series(filenames2, name='filenames'), pd.Series(labels2, name='labels2')], axis=1)

24998 24998


In [4]:
df = pd.merge(df, df2, on='filenames')

In [5]:
df

,filenames,labels,labels2
0,..\..\data_tate\A00001_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,..\..\data_tate\A00002_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
2,..\..\data_tate\A00003_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,..\..\data_tate\A00004_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
4,..\..\data_tate\A00005_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
...,...,...,...
24993,..\..\data_tate\T13856_8.jpg,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24994,..\..\data_tate\T13858_8.jpg,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24995,..\..\data_tate\T13863_8.jpg,"[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24996,..\..\data_tate\T13864_8.jpg,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [26]:
class Convolution(tf.keras.layers.Layer):
    def __init__(self, filters, shape=(224,224,3), kernel_size=3,
                 pool_size=(2,2), activation='relu', padding='same'):
        super(Convolution, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(
            input_shape = shape,
        )
        self.conv_layer = tf.keras.layers.Conv2D(
            filters = filters[0], 
            padding = padding,
            kernel_size = kernel_size,
            activation = activation,
            )
        self.conv_layer1 = tf.keras.layers.Conv2D(
            filters = filters[1],
            padding = padding,
            kernel_size = kernel_size,
            activation = activation,
            )
        self.conv_layer2 = tf.keras.layers.Conv2D(
            filters = filters[1],
            padding = padding,
            kernel_size = kernel_size,
            activation = activation,
            )
        self.nn_pooling = tf.keras.layers.MaxPooling2D(
            pool_size = pool_size,
            padding = padding,
            strides = (2,2)
            )
        self.nn_dropout = tf.keras.layers.Dropout(0.2)
        self.nn_batchnorm = tf.keras.layers.BatchNormalization(axis=-1)
        self.nn_batchnorm1 = tf.keras.layers.BatchNormalization(axis=-1)
        self.nn_batchnorm2 = tf.keras.layers.BatchNormalization(axis=-1)

        
    def call(self, input_features):
        activation = self.input_layer(input_features)
        activation = self.conv_layer(activation)
        activation = self.nn_batchnorm(activation)
        activation = self.nn_pooling(activation)
        #print(activation.shape)
        activation = self.conv_layer1(activation)
        activation = self.nn_batchnorm1(activation)
        activation = self.nn_pooling(activation)
        #print(activation.shape)
        activation = self.conv_layer2(activation)
        activation = self.nn_batchnorm2(activation)
        activation = self.nn_pooling(activation)
        #print(activation.shape)
        return activation


class FC(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(FC, self).__init__()
        self.flat = tf.keras.layers.GlobalAveragePooling2D()
        self.dense = tf.keras.layers.Dense(filters[0], activation='relu')
        self.dense1 = tf.keras.layers.Dense(filters[1], activation='relu')
        self.out = tf.keras.layers.Dense(filters[2], activation='sigmoid')

        
    def call(self, input_features):
        activation = self.flat(input_features)
        activation = self.dense(activation)
        activation = self.dense1(activation)
        activation = self.out(activation)
        return activation
    
    
class Embed(tf.keras.layers.Layer):
    def __init__(self):
        super(Embed, self).__init__()
        self.embed = tf.keras.layers.Embedding(15, 28*28, input_length=1)
        self.reshape = tf.keras.layers.Reshape((28,28))
        
        
    def call(self, input_features):
        #print(input_features.shape)
        activation = tf.argmax(input_features, axis=1)
        #print(activation.shape)
        activation = self.embed(activation)
        #print(activation.shape)
        activation = self.reshape(activation)
        return activation
    

class FullModel(tf.keras.Model):
    def __init__(self, filters_conv=[64,128], filters_dense=[2000,500,15], filters_dense1=[2000,500,141]):
        super(FullModel, self).__init__()
        self.convolution = Convolution(filters_conv)
        self.dense = FC(filters_dense)
        self.embed = Embed()
        self.dense1 = FC(filters_dense1)

    def call(self, input_features):
        activation = self.convolution(input_features)
        activation1 = self.dense(activation)
        #print(activation.shape)
        activation1 = self.embed(activation1)
        activation1 = tf.expand_dims(activation1, axis=3)
        #print(activation.shape, activation1.shape)
        activation2 = tf.concat([activation, activation1], 3)#do something that puts the conv and the embedding together
        activation2 = self.dense1(activation2)
        #print(activation2.shape)
        return activation2

class PartialModel(tf.keras.Model):
    def __init__(self, filters_conv=[64,128], filters_dense=[2000,500,15]):
        super(PartialModel, self).__init__()
        self.convolution = Convolution(filters_conv)
        self.dense = FC(filters_dense)


    def call(self, input_features):
        activation = self.convolution(input_features)
        activation1 = self.dense(activation)
        #print(activation.shape, activation1.shape)
        return activation1

model = FullModel()

partial_model = PartialModel()

model.build(input_shape=(32, 224, 224, 3))
partial_model.build(input_shape=(32, 224, 224, 3))
print(model.summary(), partial_model.summary())

Model: "full_model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution_24 (Convolution) multiple                  224512    
_________________________________________________________________
fc_36 (FC)                   multiple                  1266015   
_________________________________________________________________
embed_12 (Embed)             multiple                  11760     
_________________________________________________________________
fc_37 (FC)                   multiple                  1331141   
Total params: 2,833,428
Trainable params: 2,832,788
Non-trainable params: 640
_________________________________________________________________
Model: "partial_model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution_25 (Convolution) multiple                  224512    
_______________

In [33]:
def loss(partial_model, model, x_train, y_train, _y_train):
    loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    loss_binary = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    intermidiate_loss = loss_binary(partial_model(x_train), y_train)
    #print(intermidiate_loss)
    reconstruction_error = loss_object(model(x_train), _y_train)
    return (reconstruction_error + intermidiate_loss)/2


def train(loss, partial_model, model, opt, x_train=None, y_train=None, _y_train=None):
    with tf.GradientTape() as tape:
        gradients = tape.gradient(loss(partial_model, model, x_train, y_train, _y_train), model.trainable_variables)
        gradient_variables = zip(gradients, model.trainable_variables)
        opt.apply_gradients(gradient_variables)

In [34]:
train_x = list(df['filenames'][:18000])
train_y = list(df['labels'][:18000])
train_y_ = list(df['labels2'][:18000])
val_x = list(df['filenames'][18000:21000])
val_y = list(df['labels'][18000:21000])
val_y_ = list(df['labels2'][18000:21000])
test_x = list(df['filenames'][21000:])
test_y = list(df['labels'][21000:])
test_y_ = list(df['labels2'][21000:])

In [35]:
train_generator = create_dataset(train_x, train_y, train_y_)
val_generator = create_dataset(val_x, val_y, val_y_)
test_generator = create_dataset(test_x, test_y, test_y_)

In [ ]:
# Keep results for plotting
train_loss_results = []
train_accuracy_results = []
opt = tf.keras.optimizers.Adam() #learning_rate=learning_rate: will add in future version
num_epochs = 20

for i in range(num_epochs):
    for batch in range(len(train_x)//56):
        features, labels1, labels2 = next(iter(train_generator))
        #print(features, labels1, labels2)
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.Accuracy()
        partial_model = PartialModel()
        model = FullModel()
        train(loss, partial_model, model, opt, x_train=features, y_train=labels1, _y_train=labels2)
        loss_values = loss(partial_model, model, x_train=features, y_train=labels1, _y_train=labels2)
        print(tf.summary.scalar('loss', loss_values))
        # End epoch
        train_loss_results.append(loss_values)
        train_accuracy_results.append(epoch_accuracy(labels2, model(features, training=True)))

    if i % 2 == 0:
        print(i, epoch_loss_avg.result(), epoch_accuracy.result())